<a href="https://colab.research.google.com/github/cerr/pyCERR-Notebooks/blob/main/autosegment_CT_Lung_OARs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The pyCERR Lung CT OAR Segmentation Model

In this tutorial, we will demonstrate how to apply a pre-trained AI model to segment the OARs on a lung CT scan using pyCERR.  

## Requirements
* Python>=3.8
* Applying this model requires access to a GPU.  
  *On Colab* :  `Runtime > Change runtime type > Select GPU `

## AI model
* The segmentation model used here was trained and validated on CT scans used for RT planning. Its performance on diagnostic CTs is expected to be sub-optimal.
* The trained model is packaged as a Conda environment archive containing  python libraries and other dependencies.

  *On Colab* :  `Runtime > Change runtime type > Select GPU `

## I/O
* **Input**: DICOM-format planning lung CT scan(s).  
  
Input data should be organized as: one directory of DICOM images per patient.    
  
    
    Input dir
            |------Pat1  
                      |------img1.dcm  
                             img2.dcm  
                             ....  
                             ....  
            |-----Pat2  
                     |------img1.dcm  
                            img2.dcm  
                            ....  
                            ....  


* **Output**: DICOM RTStruct-format segmentations.


## Running the model

* The Conda archive is downloaded to a configurable `condaEnvPath`. By default `condaEnvPath = '/content/pretrainedModel/'`
* The inference script is located at   
`wrapperPath = os.path.join(condaEnvPath,'CT_LungOAR_incrMRRN/model_wrapper/run_inference_nii.py')`

* Command to execute the model
```python
!python {wrapperPath} {input_nii_directory} {output_nii_directory}
```
* [***pyCERR***](https://github.com/cerr/pyCERR) is used for data pre- and post-processing, including converting DICOM to NIfTI format, required to run the model.

## License

```
By downloading the software you are agreeing to the following terms and conditions as well as to the Terms of Use of CERR software.

**THE SOFTWARE IS PROVIDED "AS IS" AND CERR DEVELOPMENT TEAM AND ITS COLLABORATORS DO NOT MAKE ANY WARRANTY, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE, NOR DO THEY ASSUME ANY LIABILITY OR RESPONSIBILITY FOR THE USE OF THIS SOFTWARE.**

This software is for research purposes only and has not been approved for clinical use.

Software has not been reviewed or approved by the Food and Drug Administration, and is for non-clinical, IRB-approved Research Use Only. In no event shall data or images generated through the use of the Software be used in the provision of patient care.
  
YOU MAY NOT DISTRIBUTE COPIES of this software, or copies of software derived from this software, to others outside your organization without specific prior written permission from the CERR development team except where noted for specific software products.

All Technology and technical data delivered under this Agreement are subject to US export control laws and may be subject to export or import regulations in other countries. You agree to comply strictly with all such laws and regulations and acknowledge that you have the responsibility to obtain
such licenses to export, re-export, or import as may be required after delivery to you.

**You may publish papers and books using results produced using software provided that you reference the following**:
  
  * ***AI model***: https://doi.org/10.1109/TMI.2018.2857800  
```

# Downloads


## Download planning CTs (DICOM) from ***dataUrl*** to ***dataDownloadDir***


In [ ]:
import os
#workDir = r'/home/jupyter' #
workDir = r'/content' #for Colab

In [ ]:
%%capture
! pip install pyxnat
! pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git@testing"

# Download planning CT DICOM data for processing

In [ ]:
#location of input DICOM folders
inputDicomPath = os.path.join(workDir,'input')
os.makedirs(inputDicomPath, exist_ok=True)

#location of output RTSTRUCT file
outputDicomPath = os.path.join(workDir, 'output')
os.makedirs(outputDicomPath, exist_ok=True)

#temp session directory
sessionPath = os.path.join(workDir, 'session')
os.makedirs(sessionPath,exist_ok=True)


### Option 1: Download data from XNAT source (`getXNATData`)

In [ ]:
from pyxnat import Interface
from glob import glob
import urllib3
import shutil
urllib3.disable_warnings()

from pyxnat import Interface

def getXNATData(xhost,user,scandict,downloadDir):
  xnat = Interface(xhost, user, verify=False)
  os.makedirs(downloadDir, exist_ok=True)
  expdirlist = []
  for scan_entry in scandict:
    proj = scan_entry['proj']
    subj = scan_entry['subj']
    exp = scan_entry['exp']
    scan_list = scan_entry['scan_list']
    expdir = os.path.join(downloadDir,exp)
    expdirlist.append(expdir)
    os.makedirs(expdir, exist_ok = True)
    xexp = xnat.select.project(proj).subject(subj).experiment(exp)
    for scan in scan_list:
      try:
        xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('DICOM').get(downloadDir,extract=True)
      except:
        xnat.select.project(proj).subject(subj).experiment(exp).scan(scan).resource('secondary').get(downloadDir,extract=True)
    for dcmfolder in ['DICOM','secondary']:
      dcmlist = glob(os.path.join(downloadDir,dcmfolder,'*.dcm'))
      print(dcmlist)
      for dcm in dcmlist:
        shutil.move(dcm, expdir)
  for dcmfolder in ['DICOM','secondary']:
    if os.path.exists(os.path.join(downloadDir,dcmfolder)):
      os.rmdir(os.path.join(downloadDir,dcmfolder))
    if os.path.exists(os.path.join(downloadDir,dcmfolder + '.zip')):
      os.remove(os.path.join(downloadDir,dcmfolder + '.zip'))
  xnat.disconnect()
  return expdirlist

In [ ]:
xhost = 'https://xnat.yoursite.org'
user = 'usr'
scandict = [{'proj':proj,'subj':subj,'exp':exp, 'scan_list':['1']}]

folderList = getXNATData(xhost,user,scandict,inputDicomPath)

### Option 2: Download data from other HTTP source to `inputDicomDir`

In [ ]:
dataUrl = 'http://path.to/data'
dataDownloadDir = os.path.join(workDir,'tmp')
os.makedirs(dataDownloadDir,exist_ok=True)

! wget -O sampleData.gz -L {dataUrl}
! tar xf sampleData.gz -C {dataDownloadDir}
! rm sampleData.gz

# Install segmentation wrapper

### Installation of segmentation wrapper, Python environment and network weights handled by CERR-developed `model_installer`

In [ ]:
os.chdir(workDir)
!git clone https://github.com/cerr/model_installer.git
os.chdir(os.path.join(workDir,'model_installer'))
!./installer.sh -h

In [ ]:
%%capture
modelOpt = '2' #CT_LungOAR_incrMRRN
pythonOpt = 'C' #download and use pre-packaged Conda environment

! source ./installer.sh -m {modelOpt} -d {workDir} -p {pythonOpt}

In [ ]:
wrapperInstallDir = os.path.join(workDir, 'CT_LungOAR_incrMRRN')
#!git clone https://github.com/cerr/CT_LungOAR_incrMRRN {scriptInstallDir}
wrapperPath = os.path.join(wrapperInstallDir, 'model_wrapper','run_inference_nii.py')

# Location of conda archive
condaEnvDir = os.path.join(wrapperInstallDir, 'conda-pack')

# Path to activation script for Conda environment
condaActivateScript = os.path.join(condaEnvDir,'bin','activate')

# Function Definitions: Data pre- and post-processing using pyCERR

## `processInputData`: Identify the input scan and resize slices to 512x512 (pre-processing)

In [ ]:
import cerr
from cerr.dataclasses import structure
from cerr.contour import rasterseg as rs
from cerr.utils.aiPipeline import getScanNumFromIdentifier
from cerr.utils import imageProc

def processInputData(planC):

  # Identify scan index in  planC
  scanIdS = {"imageType": "CT SCAN"}
  matchScanV = getScanNumFromIdentifier(scanIdS, planC,
                                                    False)

  # Extract scan
  scanNum = matchScanV[0]
  scan3M = planC.scan[scanNum].getScanArray()
  mask3M = None

  # Resize scan and import to planC
  inputImgSizeV = np.shape(scan3M)
  gridS = planC.scan[scanNum].getScanXYZVals()
  outputImgSizeV = [512, 512, inputImgSizeV[2]]
  method = 'padorcrop3d'
  procScan3M, __, resizeGridS = imageProc.resizeScanAndMask(scan3M,
                                mask3M, gridS, outputImgSizeV, method)

  return procScan3M, resizeGridS


## `postProcAndImportSeg`: Import label maps and retain only the largest connected component to filter out false detections (post-processing).

In [ ]:
#Import label map to CERR
from cerr.utils import mask
from cerr. dataclasses import structure

def postProcAndImportSeg(outputDir,procScanNum,scanNum,planC):
  niiGlob = glob(os.path.join(outputDir,'*.nii.gz'))

  print('Importing ' + niiGlob[0]+'...')
  numStrOrig = len(planC.structure)
  planC = pc.load_nii_structure(niiGlob[0], procScanNum, planC, \
                              labels_dict = strToLabelMap)
  cpyStrNumV = np.arange(numStrOrig,len(planC.structure))
  numComponents = 1
  for label in range(numLabel):
    # Copy to original scan
    planC = structure.copyToScan(cpyStrNumV[label], scanNum, planC)
    origStr = len(planC.structure)-1
    strName =  strToLabelMap[label+1]
    # Post-process and replace input structure in planC
    procMask3M = structure.getLargestConnComps(origStr, numComponents,
                                                planC, saveFlag=True,
                                                replaceFlag=False,
                                                procSructName=strName)
  return planC

# Run segmentation: Generate OARs for all the CT scan folders located at `inputDicomPath`

In [ ]:
# Map output labels to structure names

strToLabelMap = {1:"Lung_Left", 2:"Lung_Right", 3:"Heart", 4:"Esophagus", \
                 5:"Cord", 6:"PBT"}
numLabel = len(strToLabelMap)

In [ ]:
%%capture
import subprocess
import numpy as np
import cerr
from cerr import plan_container as pc
from cerr.dataclasses import scan as cerrScn
from cerr.utils.aiPipeline import createSessionDir
from cerr.dcm_export import rtstruct_iod

# Loop over pyCERR files
folderList = glob(os.path.join(inputDicomPath,'*'))
modality = 'CT'
scanNum = 0

for dcmDir in folderList:
    fname = os.path.basename(dcmDir)
    # Create session dir to store temporary data
    modInputPath, modOutputPath = createSessionDir(sessionPath, dcmDir)


    # Import DICOM scan to planC
    planC = pc.load_dcm_dir(dcmDir)
    numExistingStructs = len(planC.structure)

    # Pre-process data
    procScan3M, resizeGridS = processInputData(planC)
    planC = pc.import_scan_array(procScan3M, resizeGridS[0], \
            resizeGridS[1], resizeGridS[2], modality, scanNum, planC)
    procScanNum = len(planC.scan) - 1

    # Export inputs to NIfTI
    scanFilename = os.path.join(modInputPath,
                                f"{fname}_scan_3D.nii.gz")
    planC.scan[procScanNum].save_nii(scanFilename)

    # Apply model
    subprocess.run(f"source {condaActivateScript} && python {wrapperPath} \
                  {modInputPath} {modOutputPath}", \
                  capture_output=False,shell=True,executable="/bin/bash")

    # Import results to planC
    planC = postProcAndImportSeg(modOutputPath,procScanNum,scanNum,planC)
    newNumStructs = len(planC.structure)

    # Export segmentations to DICOM
    structFileName = fname +'_AI_seg_RTSTRUCT.dcm'
    structFilePath = os.path.join(outputDicomPath,structFileName)
    structNumV = np.arange(numExistingStructs,newNumStructs)
    indOrigV = np.array([cerrScn.getScanNumFromUID(planC.structure[structNum].assocScanUID,\
                        planC) for structNum in structNumV], dtype=int)
    origIndsToExportV = structNumV[indOrigV == scanNum]
    seriesDescription = "AI Generated"
    exportOpts = {'seriesDescription': seriesDescription}
    rtstruct_iod.create(structsToExportV,structFilePath,planC,exportOpts)

## **Optional**: Uncomment the following to download the output segmentations to your workspace bucket.

In [ ]:
# workspaceBucket = os.environ['WORKSPACE_BUCKET']
# !gcloud storage cp -r {outputDicomPath} {workspaceBucket}

# Display results

## Overlay AI segmentations on scan for visualization using ***Matplotlib***

Note: This example displays the last segmented dataset by default.  
Load the appropriate pyCERR archive to `planC` to view results for desired dataset.

In [ ]:
from cerr.viewer import showMplNb

showMplNb(scanNum, origIndsToExportV, planC,\
          windowCenter=-400, windowWidth=2000)